In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy
import sys
from glob import glob

In [3]:
from pyuoi.linear_model.var import VAR
from sklearn.model_selection import KFold

In [4]:
sys.path.append('../..')

In [5]:
from em import ARMAStateSpaceML
from loaders import load_sabes
from dca.cov_util import calc_cross_cov_mats_from_data
from neurosim.models.var import VAR as VARss
from neurosim.models.var import form_companion
from neurosim.models.ssr import StateSpaceRealization as SSR

In [6]:
# On *Sabes* Data, demonstrate that fitting ARMAStateSpace initialized by VAR improves cross-validated CCM prediction performance --> this is actually too large to do locally. Instead go back to synthetic model

In [7]:
state_dim = 30
obs_dim = 20
A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))
while max(np.abs(np.linalg.eigvals(A))) > 0.99:
    A = np.random.normal(scale=1/(1.7 * np.sqrt(state_dim)), size=(state_dim, state_dim))

C = scipy.stats.ortho_group.rvs(state_dim)[:, 0:obs_dim].T
ssr = SSR(A=A, B=np.eye(A.shape[0]), C=C)
ccm = ssr.autocorrelation(5)

In [8]:
y = ssr.trajectory(int(5e3))

In [9]:
cv_split = list(KFold(5).split(y))

In [10]:
train_idxs, test_idxs = cv_split[0]

In [11]:
# Fit OLS VAR(3)
varmodel = VAR(order=2, estimator='ols')

In [12]:
varmodel.fit(y[train_idxs])

Rows of VAR matrix processed: 20it [00:00, 631.22it/s]


In [13]:
varss = VARss(varmodel.coef_)

In [14]:
ccm_test = calc_cross_cov_mats_from_data(y[test_idxs], 5)
ccm_var = varss.autocorrelation(5)

In [15]:
ccm_var_error = np.array([np.linalg.norm(ccm_var[i] - ccm_test[i]) for i in range(5)])

In [16]:
ccm_var_error

array([1.3704781 , 1.30507517, 1.12619335, 1.07052301, 0.99779408])

In [17]:
varmaxmodel = ARMAStateSpaceML(init_strategy='manual')

In [18]:
# To initiate parameters, solve min phase
varss.solve_min_phase()

In [19]:
varmaxmodel.fit(y[train_idxs], state_dim = 30, Ainit=varss.A, Cinit=varss.C, Kinit=varss.Bmin @ np.linalg.inv(varss.Dmin), Rinit= varss.Dmin @ varss.Dmin.T, x0 = np.zeros(varss.A.shape[0]), Sigma0 = varss.P, Qinit=varss.Bmin @ varss.Bmin.T)

../../em.py:44: ComplexWarning: Casting complex values to real discards the imaginary part
  Psqrt_pred[i] = Psqrt_pred_
../../em.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
  Ppred[i] = Psqrt_pred_ @ Psqrt_pred_.T
../../em.py:59: ComplexWarning: Casting complex values to real discards the imaginary part
  Psqrt_filt[i] = Psqrt_filt_
../../em.py:88: ComplexWarning: Casting complex values to real discards the imaginary part
  Psmooth[i] = Psqrt_smooth @ Psqrt_smooth.T
../../em.py:358: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A.transpose(-2, -1).conj()).transpose(-2, -1).conj() (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:1284.)
  self.

E step: 6.877015


  0%|          | 0/100 [00:00<?, ?it/s]/home/akumar/anaconda3/envs/dyn/lib/python3.7/site-packages/torch/functional.py:1403: UserWarning: torch.chain_matmul is deprecated and will be removed in a future PyTorch release. Use torch.linalg.multi_dot instead, which accepts a list of two or more tensors rather than multiple parameters. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/LinearAlgebra.cpp:697.)
  return _VF.chain_matmul(matrices)  # type: ignore[attr-defined]
100%|██████████| 100/100 [02:14<00:00,  1.35s/it]


M step: 138.856035
E step: 6.669816


100%|██████████| 100/100 [02:15<00:00,  1.35s/it]


...

M step: 134.722640
E step: 6.258533


100%|██████████| 100/100 [02:04<00:00,  1.25s/it]


M step: 129.210367
E step: 6.366287


100%|██████████| 100/100 [02:07<00:00,  1.28s/it]


M step: 132.367291


In [23]:
np.abs(np.linalg.eigvals(A))

array([0.57277005, 0.57277005, 0.52274874, 0.5059852 , 0.5059852 ,
       0.53413055, 0.53413055, 0.47570937, 0.47570937, 0.50046757,
       0.50046757, 0.39653482, 0.39653482, 0.39740329, 0.39740329,
       0.34455306, 0.34455306, 0.28284213, 0.38452126, 0.38452126,
       0.46954518, 0.43630209, 0.30691375, 0.26642725, 0.26642725,
       0.20581297, 0.20581297, 0.09976788, 0.09976788, 0.06514524])

In [22]:
np.abs(np.linalg.eigvals(varmaxmodel.A))

array([0.5727169 , 0.5727169 , 0.54088318, 0.54088318, 0.5200568 ,
       0.5200568 , 0.47115094, 0.48527573, 0.48527573, 0.46942597,
       0.46942597, 0.40369581, 0.40369581, 0.3418304 , 0.3418304 ,
       0.36707684, 0.36707684, 0.25180371, 0.25180371, 0.2888591 ,
       0.2888591 , 0.20053664, 0.38009606, 0.3239354 , 0.3239354 ,
       0.20064043, 0.20064043, 0.2961741 , 0.28816883, 0.28816883,
       0.23834564, 0.23834564, 0.18020215, 0.18020215, 0.19134532,
       0.19134532, 0.09786041, 0.09786041, 0.04497333, 0.04497333])

In [24]:
varmaxmodel.Q

array([[ 2.59725650e-01,  1.35558770e-01,  9.03894822e-02, ...,
        -1.24578375e-02,  2.38846074e-01,  1.36475029e-01],
       [ 1.35558770e-01,  2.89426768e-01,  6.04160904e-02, ...,
         6.46501187e-02,  1.92544170e-01,  2.73604728e-02],
       [ 9.03894822e-02,  6.04160904e-02,  1.86769343e-01, ...,
        -1.99227919e-02,  1.30371678e-01, -8.80720099e-02],
       ...,
       [-1.24578375e-02,  6.46501187e-02, -1.99227919e-02, ...,
         1.02549292e+00, -5.56532157e-03,  7.46308756e-03],
       [ 2.38846074e-01,  1.92544170e-01,  1.30371678e-01, ...,
        -5.56532157e-03,  1.02162045e+00, -6.97240376e-04],
       [ 1.36475029e-01,  2.73604728e-02, -8.80720099e-02, ...,
         7.46308756e-03, -6.97240376e-04,  1.03280567e+00]])

In [25]:
def autocorrelation(A, C, K, R, T):

    autocorr = np.zeros((T, C.shape[0], C.shape[0]))

    P = scipy.linalg.solve_discrete_lyapunov(A, K @ R @ K.T)

    autocorr[0, ...] = C @ P @ C.T + R

    # Construct Cbar from A, P, C, and S
    Cbar = C @ P @ A.T + (K @ R).T

    for i in range(1, T):
        autocorr[i, ...] = C @ np.linalg.matrix_power(A, i - 1) @ Cbar.T
    return autocorr  

In [26]:
ccm_varmax = autocorrelation(varmaxmodel.A, varmaxmodel.C, varmaxmodel.K, varmaxmodel.R, 5)

In [27]:
ccm_varmax_error = np.array([np.linalg.norm(ccm_varmax[i] - ccm_test[i]) for i in range(5)])

In [28]:
ccm_varmax_error

array([4.48239783, 2.3785857 , 1.72001859, 1.1860499 , 1.06047833])

In [29]:
ccm_var_error

array([1.3704781 , 1.30507517, 1.12619335, 1.07052301, 0.99779408])